In [1]:
import pandas as pd
import numpy as np

In [2]:
tree_test = pd.read_csv("/Users/rezadoobary/Documents/ML-correlator/Misclassifications/data/test_tree.csv")
gnn_test = "/Users/rezadoobary/Downloads/test_predictions_loop_11 (2).csv"
gnn_test = pd.read_csv(gnn_test)


tree_train = pd.read_csv("/Users/rezadoobary/Documents/ML-correlator/Misclassifications/data/train_tree.csv")
gnn_train = "/Users/rezadoobary/Downloads/train_predictions_loop_['7', '8', '9', '10'].csv"
gnn_train = pd.read_csv(gnn_train)

In [3]:
from sklearn.metrics import roc_auc_score

roc_auc_score(gnn_test['y_true'].values, gnn_test['y_pred'].values)

0.9816234507082789

In [4]:
roc_auc_score(gnn_test['y_true'].values, tree_test['predicted'].values)

0.8568277392499816

In [5]:
df_train = pd.DataFrame({'gnn':gnn_train['y_pred'].values, 
 'tree':tree_train['predicted'].values, 'true':gnn_train['y_true'].values})

df_test = pd.DataFrame({'gnn':gnn_test['y_pred'].values, 
 'tree':tree_test['predicted'].values, 'true':gnn_test['y_true'].values})

In [6]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(df_train[['gnn','tree']], df_train[['true']])

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [7]:
preds = model.predict_proba(df_test[['gnn','tree']])[:,1]

In [8]:
roc_auc_score(df_test['true'].values, preds)

0.968297691547678

In [9]:
# overlay model doesn't appear to help

In [10]:
# learning one from the other

In [13]:
res = []
for l in [7,8,9,10]:
    temp = pd.read_csv(f'/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/{l}loopfeats.csv')
    temp['loop'] = l
    res.append(temp)
df_feats = pd.concat(res)

In [14]:
df_train['loop'] = df_feats['loop'].values

In [15]:
for l in [7,8,9,10]:
    res = df_train[df_train['loop'] == l]
    print(roc_auc_score(res['true'],res['gnn']),roc_auc_score(res['true'],res['tree']))

0.9875389408099688 0.9965568125922283
1.0 0.9706236450875828
0.9999931772635561 0.9465073870150063
0.9998665749754463 0.9420772801167445


In [16]:
res = df_train[df_train['loop'] == 9]
roc_auc_score(res['true'],res['gnn'])

0.9999931772635561

In [75]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

def find_rows_to_remove_for_perfect_auc(df, label_col="true", score_col="gnn"):
    """
    Returns the minimal set of row indices (from df) to remove so that
    all remaining negatives have score <= all remaining positives (strictly: pos > neg).
    This yields ROC AUC = 1.0, provided at least one pos and one neg remain.
    """
    if set(df[label_col].unique()) - {0, 1}:
        raise ValueError(f"{label_col} must be binary 0/1.")

    # Keep original index to report back which rows to remove
    work = df[[label_col, score_col]].copy()
    work["__orig_idx__"] = df.index

    # Sort by score ascending
    work = work.sort_values(score_col, kind="mergesort").reset_index(drop=True)

    y = work[label_col].to_numpy().astype(int)
    n = len(work)
    pos_total = int(y.sum())
    neg_total = int(n - pos_total)

    if pos_total == 0 or neg_total == 0:
        # Already degenerate (AUC undefined). Nothing sensible to do.
        return {
            "remove_indices": [],
            "cut_position": None,
            "cut_score": None,
            "kept_counts": {"neg_left": neg_total, "pos_right": pos_total},
            "removals_count": 0,
            "auc_after": None,
            "note": "Only one class present; ROC AUC is undefined."
        }

    # Prefix counts over sorted order
    pos_prefix = np.zeros(n + 1, dtype=int)  # pos in first k
    neg_prefix = np.zeros(n + 1, dtype=int)  # neg in first k
    for k in range(1, n + 1):
        pos_prefix[k] = pos_prefix[k - 1] + y[k - 1]
        neg_prefix[k] = k - pos_prefix[k]

    # Evaluate all cut positions k = 0..n
    # Left = [0..k-1], Right = [k..n-1]
    # Kept if: negatives on left + positives on right
    kept = neg_prefix + (pos_total - pos_prefix)

    # We also want ROC AUC to be *defined*, so enforce at least one neg on left AND one pos on right
    valid = (neg_prefix > 0) & ((pos_total - pos_prefix) > 0)
    if not valid.any():
        # Fall back to best unconstrained split, but warn that AUC may be undefined
        k_star = int(kept.argmax())
        auc_defined = False
    else:
        k_star = int(np.where(kept == kept[valid].max())[0][0])  # first max among valid
        auc_defined = True

    # Identify removals under the chosen cut
    # Remove positives on the left and negatives on the right
    left_mask = np.arange(n) < k_star
    right_mask = ~left_mask
    remove_left_pos = left_mask & (y == 1)
    remove_right_neg = right_mask & (y == 0)
    to_remove_mask = remove_left_pos | remove_right_neg

    remove_indices = work.loc[to_remove_mask, "__orig_idx__"].tolist()
    removals_count = len(remove_indices)

    # Derive a representative cut score (for reference)
    if k_star == 0:
        cut_score = work.loc[0, score_col] - np.finfo(float).eps
    elif k_star == n:
        cut_score = work.loc[n - 1, score_col] + np.finfo(float).eps
    else:
        s_left = work.loc[k_star - 1, score_col]
        s_right = work.loc[k_star, score_col]
        cut_score = (float(s_left) + float(s_right)) / 2.0

    # Verify ROC AUC on the remaining set (if both classes remain)
    kept_df = df.drop(index=remove_indices)
    auc_after = None
    if kept_df[label_col].nunique() == 2:
        auc_after = roc_auc_score(kept_df[label_col], kept_df[score_col])

    return {
        "remove_indices": remove_indices,
        "cut_position": k_star,
        "cut_score": cut_score,
        "kept_counts": {
            "neg_left": int(neg_prefix[k_star]),
            "pos_right": int(pos_total - pos_prefix[k_star])
        },
        "removals_count": int(removals_count),
        "auc_after": auc_after,
        "note": None if auc_defined else "Best split keeps only one class; ROC AUC would be undefined."
    }

# ---------- Usage with your subset ----------
res = df_train[df_train['loop'] == 7]  # your filter
# Optionally reset/drop nothing; original indices are preserved for reporting.
out = find_rows_to_remove_for_perfect_auc(res, label_col="true", score_col="gnn")

print("Rows to remove (original indices):", out["remove_indices"])
print("Number to remove:", out["removals_count"])
print("Cut score (reference):", out["cut_score"])
print("Kept counts  -> negatives on left / positives on right:", out["kept_counts"])
print("AUC after removal:", out["auc_after"])
if out["note"]:
    print("Note:", out["note"])


Rows to remove (original indices): [111, 127, 88, 113, 129, 40, 134, 159, 109]
Number to remove: 9
Cut score (reference): 0.524985925
Kept counts  -> negatives on left / positives on right: {'neg_left': 53, 'pos_right': 102}
AUC after removal: 1.0


In [76]:
df_feats[df_feats['loop'] == 7].iloc[out["remove_indices"]]

,COEFFICIENTS,Basic_num_nodes,Basic_num_edges,Basic_min_degree,Basic_max_degree,Basic_avg_degree,Basic_degree_std,Basic_degree_skew,Basic_density,Basic_edge_to_node_ratio,...,Spectral_lap_eig_8,Spectral_lap_eig_9,Kirchhoff_index,Planarity_num_faces,Planarity_face_size_mean,Planarity_face_size_max,Symmetry_automorphism_group_order,Symmetry_num_orbits,Symmetry_orbit_size_max,loop
111,1,11,22,4,4,4.000000,0.000000,NaN,0.400000,2.000000,...,5.730449,6.000000,31.222760,13,3.384615,5,2,7,2,7
127,1,11,24,4,5,4.363636,0.481046,0.566947,0.436364,2.181818,...,5.732051,6.815423,27.518794,15,3.200000,4,2,8,2,7
88,1,11,23,4,5,4.181818,0.385695,1.649916,0.418182,2.090909,...,5.788547,6.176451,30.119933,14,3.285714,5,2,6,2,7
113,1,11,24,4,5,4.363636,0.481046,0.566947,0.436364,2.181818,...,6.000000,6.626212,28.139394,15,3.200000,4,2,8,2,7
129,1,11,23,4,5,4.181818,0.385695,1.649916,0.418182,2.090909,...,5.634868,6.675131,28.888774,14,3.285714,4,2,6,2,7
40,0,11,26,4,6,4.727273,0.616575,0.250010,0.472727,2.363636,...,6.586290,7.152246,25.066375,17,3.058824,4,1,11,1,7
134,0,11,24,4,5,4.363636,0.481046,0.566947,0.436364,2.181818,...,6.453263,6.618034,27.411832,15,3.200000,4,1,11,1,7
159,0,11,25,4,6,4.545455,0.782030,0.989676,0.454545,2.272727,...,6.302776,7.386577,26.789631,16,3.125000,4,2,6,2,7
109,0,11,26,4,6,4.727273,0.749656,0.492205,0.472727,2.363636,...,6.746920,7.387135,25.202285,17,3.058824,4,1,11,1,7


In [66]:
df_feats[df_feats['loop'] == 9].groupby('Symmetry_automorphism_group_order')['COEFFICIENTS'].mean()

Symmetry_automorphism_group_order
1     0.395431
2     0.501484
3     0.000000
4     0.398936
6     1.000000
8     0.185185
16    0.000000
32    0.000000
44    1.000000
Name: COEFFICIENTS, dtype: float64

In [33]:
df_feats['Basic_num_nodes'].value_counts()

Basic_num_nodes
14    153252
13     13972
12      1432
11       164
Name: count, dtype: int64

In [34]:
df_feats.iloc[[i for i in np.arange(0,len(df_feats)) if i not in out["remove_indices"]]].describe()

,COEFFICIENTS,Basic_num_nodes,Basic_num_edges,Basic_min_degree,Basic_max_degree,Basic_avg_degree,Basic_degree_std,Basic_degree_skew,Basic_density,Basic_edge_to_node_ratio,...,Spectral_lap_eig_8,Spectral_lap_eig_9,Kirchhoff_index,Planarity_num_faces,Planarity_face_size_mean,Planarity_face_size_max,Symmetry_automorphism_group_order,Symmetry_num_orbits,Symmetry_orbit_size_max,loop
count,168806.000000,168806.000000,168806.000000,168806.000000,168806.000000,168806.000000,168806.000000,168697.000000,168806.000000,168806.000000,...,168806.000000,168806.000000,168806.000000,168806.000000,168806.000000,168806.000000,168806.000000,168806.000000,168806.000000,168806.000000
mean,0.342132,13.897433,32.321108,4.000012,6.342843,4.651119,0.793805,0.889792,0.360876,2.325560,...,5.637433,6.034973,46.708019,20.423676,3.169758,4.283420,1.111501,13.577266,1.080459,9.897433
std,0.474425,0.338886,1.557453,0.003442,0.764874,0.187698,0.202274,0.455492,0.017444,0.093849,...,0.279163,0.283860,4.386231,1.400301,0.075656,0.527928,0.906941,1.386534,0.341351,0.338886
min,0.000000,11.000000,22.000000,4.000000,4.000000,4.000000,0.000000,-2.041241,0.307692,2.000000,...,4.554958,5.000000,23.571138,13.000000,3.000000,3.000000,1.000000,1.000000,1.000000,7.000000
25%,0.000000,14.000000,31.000000,4.000000,6.000000,4.571429,0.622700,0.556197,0.351648,2.285714,...,5.444155,5.846817,44.691564,20.000000,3.100000,4.000000,1.000000,14.000000,1.000000,10.000000
50%,0.000000,14.000000,32.000000,4.000000,6.000000,4.714286,0.795395,0.859894,0.362637,2.357143,...,5.625127,6.000000,46.682620,20.000000,3.142857,4.000000,1.000000,14.000000,1.000000,10.000000
75%,1.000000,14.000000,33.000000,4.000000,7.000000,4.769231,0.914732,1.159153,0.373626,2.384615,...,5.829393,6.208981,48.820240,21.000000,3.200000,5.000000,1.000000,14.000000,1.000000,10.000000
max,1.000000,14.000000,36.000000,5.000000,12.000000,5.142857,2.799417,2.404129,0.490909,2.571429,...,7.342923,9.000000,115.883848,24.000000,3.500000,9.000000,120.000000,14.000000,14.000000,10.000000


In [22]:
df_feats.shape

(168820, 58)

In [11]:
df_train['diff'] = np.abs(df_train['gnn'] - df_train['tree'])
df_train['tg_diff'] = df_train['tree'] - df_train['gnn']
df_train['gt_diff'] = df_train['gnn'] - df_train['tree']

In [ ]:
case = df_train.sort_values('tg_diff')aaa[::-1]
remove0 = case[(case['true'] == 1) & (case['tg_diff'] > 0.1)].sort_values('tg_diff')[::-1].index

In [99]:
case = df_train.sort_values('gt_diff')[::-1]
remove1 = case[(case['true'] == 0) & (case['gt_diff'] > 0.1)].sort_values('gt_diff')[::-1].index

In [100]:
df_train_ = df_train.drop(remove0.values).drop(remove1.values) # removes rows with indices 2, 5, 10

In [101]:
roc_auc_score(df_train_['true'],df_train_['gnn'])

0.9999779075658359

In [102]:
roc_auc_score(df_train['true'],df_train['gnn'])

0.99988355084821

In [105]:
df_train['loop'] = tree_train['source_loop'].values

0.9875389408099688 0.9965568125922283
1.0 0.9706236450875828
0.9999931772635561 0.9465073870150063
0.9998665749754463 0.9420772801167445


In [43]:
trues = df_train[(df_train['true'] == 1) & (df_train['tree'] > df_train['gnn'])]
trues

,gnn,tree,true
10,0.760596,0.784117,1
11,0.552906,0.943130,1
36,0.803408,0.915372,1
50,0.869189,0.911977,1
54,0.904002,0.968507,1
...,...,...,...
154660,0.807484,0.866866,1
155002,0.434962,0.455878,1
155182,0.472638,0.643457,1
162970,0.367086,0.519734,1


In [44]:
trues

,gnn,tree,true
10,0.760596,0.784117,1
11,0.552906,0.943130,1
36,0.803408,0.915372,1
50,0.869189,0.911977,1
54,0.904002,0.968507,1
...,...,...,...
154660,0.807484,0.866866,1
155002,0.434962,0.455878,1
155182,0.472638,0.643457,1
162970,0.367086,0.519734,1


In [42]:
falses = df_train[df_train['true'] == 0]
falses[falses['tree'] < falses['gnn']]

,gnn,tree,true
15,0.438258,3.863212e-01,0
38,0.517558,2.990370e-01,0
40,0.544266,4.924832e-01,0
56,0.205551,1.587940e-01,0
79,0.420757,2.072338e-01,0
...,...,...,...
165877,0.029810,1.980200e-03,0
165974,0.000984,8.066511e-04,0
166230,0.013956,2.316924e-03,0
167170,0.000001,4.496355e-07,0
